# This notebook is for the 30 Days of Machine Learning contest on Kaggle.

#### This notebook was created by following Abhishek Thakur's YouTube video tutorial: 
https://www.youtube.com/watch?v=m5YSKPMjkrk&list=PL98nY_tJQXZnP-k3qCDd1hljVSciDV9_N&index=22


In [3]:
# Imports
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import optuna

In [4]:
df = pd.read_csv("data/train_folds.csv")
df_test = pd.read_csv("data/test.csv")
sample_submission = pd.read_csv("data/sample_submission.csv")

useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
df_test = df_test[useful_features]

for col in object_cols:
    temp_df = []
    temp_test_feat = None
    for fold in range(5):
        xtrain = df[df.kfold != fold].reset_index(drop=True)
        xvalid = df[df.kfold == fold].reset_index(drop=True)
        feat = xtrain.groupby(col)["target"].agg("mean")
        feat = feat.to_dict()
        xvalid.loc[:, f"tar_enc_{col}"] = xvalid[col].map(feat)
        temp_df.append(xvalid)
        if temp_test_feat is None:
            temp_test_feat = df_test[col].map(feat)
        else:
            temp_test_feat += df_test[col].map(feat)
            
    temp_test_feat /= 5
    df_test.loc[:, f"tar_enc_{col}"] = temp_test_feat
    df = pd.concat(temp_df)
    
useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if col.startswith("cat")]
df_test = df_test[useful_features]


In [6]:
def run(trial):
    fold = 0
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 0.25, log=True)
    reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-8, 100.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-8, 100.0)
    subsample = trial.suggest_float("subsample", 0.1, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
    max_depth = trial.suggest_int("max_depth", 1, 7)

    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)

    ytrain = xtrain.target
    yvalid = xvalid.target

    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]

    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])

    model = XGBRegressor(
        random_state=42,
        tree_method='gpu_hist',
        gpu_id=1,
        predictor="gpu_predictor",
        n_estimators=7000,
        learning_rate=learning_rate,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        max_depth=max_depth,
    )
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    preds_valid = model.predict(xvalid)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    return rmse

In [9]:
study = optuna.create_study(direction="minimize")
study.optimize(run, n_trials=100) #100

[I 2021-08-28 14:18:21,585] A new study created in memory with name: no-name-f860ef5d-4f5b-4365-a9b8-2d800a8adb46


[14:18:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:231: Only 1 GPUs are visible, setting `gpu_id` to 0
[0]	validation_0-rmse:7.38048
[593]	validation_0-rmse:0.72727


[I 2021-08-28 14:18:32,885] Trial 0 finished with value: 0.7267521796706935 and parameters: {'learning_rate': 0.05201948991047788, 'reg_lambda': 0.787402674291422, 'reg_alpha': 0.015184911247291606, 'subsample': 0.12870670332651485, 'colsample_bytree': 0.9810661672413583, 'max_depth': 6}. Best is trial 0 with value: 0.7267521796706935.


[14:18:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:231: Only 1 GPUs are visible, setting `gpu_id` to 0
[0]	validation_0-rmse:7.15341
[1000]	validation_0-rmse:0.72008
[2000]	validation_0-rmse:0.71927
[2271]	validation_0-rmse:0.71942


[I 2021-08-28 14:18:54,215] Trial 1 finished with value: 0.7192412616095979 and parameters: {'learning_rate': 0.08132642869924218, 'reg_lambda': 0.1192198503116406, 'reg_alpha': 0.018251407941978978, 'subsample': 0.3810489714698997, 'colsample_bytree': 0.24771338187237152, 'max_depth': 3}. Best is trial 1 with value: 0.7192412616095979.


...

[I 2021-08-28 16:03:19,904] Trial 97 finished with value: 0.7182383268668046 and parameters: {'learning_rate': 0.013510702781057494, 'reg_lambda': 1.2127582966585337, 'reg_alpha': 0.08878333184942185, 'subsample': 0.8149394456268888, 'colsample_bytree': 0.2474961891447045, 'max_depth': 4}. Best is trial 61 with value: 0.7175232333970853.


[16:03:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:231: Only 1 GPUs are visible, setting `gpu_id` to 0
[0]	validation_0-rmse:7.63629
[1000]	validation_0-rmse:0.72666
[2000]	validation_0-rmse:0.72279
[3000]	validation_0-rmse:0.72077
[4000]	validation_0-rmse:0.71965
[5000]	validation_0-rmse:0.71894
[6000]	validation_0-rmse:0.71853
[6999]	validation_0-rmse:0.71829


[I 2021-08-28 16:04:35,527] Trial 98 finished with value: 0.7182887666707375 and parameters: {'learning_rate': 0.01862877047002312, 'reg_lambda': 0.05148866777173494, 'reg_alpha': 2.655407647612209e-05, 'subsample': 0.7836242332685336, 'colsample_bytree': 0.27365310146938804, 'max_depth': 3}. Best is trial 61 with value: 0.7175232333970853.


[16:04:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:231: Only 1 GPUs are visible, setting `gpu_id` to 0
[0]	validation_0-rmse:7.64557
[1000]	validation_0-rmse:0.72487
[2000]	validation_0-rmse:0.72091
[3000]	validation_0-rmse:0.71932
[4000]	validation_0-rmse:0.71855
[5000]	validation_0-rmse:0.71813
[6000]	validation_0-rmse:0.71791
[6999]	validation_0-rmse:0.71780


[I 2021-08-28 16:06:05,360] Trial 99 finished with value: 0.7177968841729827 and parameters: {'learning_rate': 0.017423971356174236, 'reg_lambda': 0.02901577751677582, 'reg_alpha': 1.0987563320907057, 'subsample': 0.8760175622799714, 'colsample_bytree': 0.144028236396453, 'max_depth': 4}. Best is trial 61 with value: 0.7175232333970853.


In [10]:
study.best_params

{'learning_rate': 0.018728063145787083,
 'reg_lambda': 0.01636958273385703,
 'reg_alpha': 2.0005220900285042e-08,
 'subsample': 0.8948010011350874,
 'colsample_bytree': 0.12936159520820203,
 'max_depth': 4}

In [14]:
final_predictions = []
scores = []
for fold in range(5):
    
    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()
    
    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])

    model = XGBRegressor(
        random_state=fold,
        tree_method='gpu_hist',
        gpu_id=0,
        predictor="gpu_predictor",
        n_estimators=7000,
        learning_rate=0.018728063145787083,
        reg_lambda=0.01636958273385703,
        reg_alpha=2.0005220900285042e-08,
        subsample=0.8948010011350874,
        colsample_bytree=0.12936159520820203,
        max_depth=4,
    )
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)
    
print(np.mean(scores), np.std(scores))

0 0.7177449643775529
1 0.7173084385848599
2 0.719185160034629
3 0.7230672312764698
4 0.7417915842242924
0.7238194756995607 0.009212304058043765


In [15]:
preds = np.mean(np.column_stack(final_predictions), axis=1)

In [16]:
sample_submission.target = preds
sample_submission.to_csv("submission3.csv", index=False)